In [1]:
import mxnet as mx
from mxnet import gluon
from mxnet.gluon import nn
from mxnet import autograd as ag
from mxnet import ndarray as nd
from Algorithm.CNN import CNN_Model
from Algorithm.MLP import MLP
from Tools import utils
import numpy as np

In [2]:
ctx = utils.try_all_gpus()
mlp = MLP()
lenet = CNN_Model('LeNet')


def get_data(data_idx, batch_size):
    data = "E:\\PythonProjects\\Mxnet_FederatedLearning\\Fed_Client\\FedAvg_data_random\\train_data" + str(data_idx) + ".npy"
    label = "E:\\PythonProjects\\Mxnet_FederatedLearning\\Fed_Client\\FedAvg_data_random\\train_label" + str(data_idx) + ".npy"
    data = np.load(data)
    label = np.load(label)
    train_data = mx.io.NDArrayIter(data,label,batch_size=batch_size,shuffle=True)
    return train_data

def validate(net):
    mnist = mx.test_utils.get_mnist()
    val_data = mx.io.NDArrayIter(mnist['test_data'],mnist['test_label'],batch_size=100)
    metric = mx.metric.Accuracy()
    val_data.reset()
    for batch in val_data:
        data = gluon.utils.split_and_load(batch.data[0], ctx_list=ctx, batch_axis=0)
        label = gluon.utils.split_and_load(batch.label[0], ctx_list=ctx, batch_axis=0)
        outputs = []
        for x in data:
            outputs.append(net(x))
        metric.update(label, outputs)
    name,acc =metric.get()
    return acc

In [3]:
def train_till_acc(net, stop_acc, learning_rate, batch_size, epoch):
    round = 0
    for client in range(1000):
        client = client%100
        train_data = get_data(client, batch_size)
        softmax_cross_entropy_loss = gluon.loss.SoftmaxCrossEntropyLoss()
        trainer = trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': learning_rate})
        metric = mx.metric.Accuracy()
        stop_flag = False
        print("Round %d starting"%(round))
        for i in range(epoch):
            train_data.reset()
            for batch in train_data:
                data = gluon.utils.split_and_load(batch.data[0], ctx_list=ctx, batch_axis=0)
                label = gluon.utils.split_and_load(batch.label[0], ctx_list=ctx, batch_axis=0)
                outputs = []
                with ag.record():
                    for x, y in zip(data, label):
                        z = net(x)
                        loss = softmax_cross_entropy_loss(z, y)
                        loss.backward()
                        outputs.append(z)
                metric.update(label, outputs)
                trainer.step(batch.data[0].shape[0])
            name, acc = metric.get()
            metric.reset()
            print('training acc at epoch %d: %s=%f'%(i, name, acc))
        acc = validate(net)
        print("accuracy in test set %f"%(acc))
        if acc >= stop_acc:
            break
        round += 1
    print("round: ",round)

In [4]:
mlp.load_parameters("mlp.params",ctx=ctx)
train_till_acc(mlp,0.93,0.02,100,5)

800
Round 7 starting
training acc at epoch 0: accuracy=0.830000
training acc at epoch 1: accuracy=0.843333
training acc at epoch 2: accuracy=0.846667
training acc at epoch 3: accuracy=0.856667
training acc at epoch 4: accuracy=0.871667
accuracy in test set 0.845100
Round 8 starting
training acc at epoch 0: accuracy=0.850000
training acc at epoch 1: accuracy=0.855000
training acc at epoch 2: accuracy=0.865000
training acc at epoch 3: accuracy=0.855000
training acc at epoch 4: accuracy=0.866667
accuracy in test set 0.858800
Round 9 starting
training acc at epoch 0: accuracy=0.878333
training acc at epoch 1: accuracy=0.886667
training acc at epoch 2: accuracy=0.890000
training acc at epoch 3: accuracy=0.893333
training acc at epoch 4: accuracy=0.895000
accuracy in test set 0.854000
Round 10 starting
training acc at epoch 0: accuracy=0.895000
training acc at epoch 1: accuracy=0.911667
training acc at epoch 2: accuracy=0.913333
training acc at epoch 3: accuracy=0.920000
training acc at epoc